In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

df= pd.read_csv('C://Users//User//Desktop//MSc Westminster//Dissertation//DataSets//Heart_Disease_Indicators.csv')

In [2]:
from sklearn.model_selection import train_test_split

target_size = 50000

num_class_0 = int(target_size * df['HeartDiseaseorAttack'].value_counts(normalize=True)[0])
num_class_1 = target_size - num_class_0

df_class_0 = df[df['HeartDiseaseorAttack'] == 0]
df_class_1 = df[df['HeartDiseaseorAttack'] == 1]

df_class_0_sampled = df_class_0.sample(n=num_class_0, random_state=15)
df_class_1_sampled = df_class_1.sample(n=num_class_1, random_state=15)

df_sampled = pd.concat([df_class_0_sampled, df_class_1_sampled])

df_sampled = df_sampled.sample(frac=1, random_state=15).reset_index(drop=True)
df= df_sampled
print(df['HeartDiseaseorAttack'].value_counts(normalize=True))

0    0.90582
1    0.09418
Name: HeartDiseaseorAttack, dtype: float64


In [3]:
df.drop_duplicates(inplace= True)
df.shape

(48050, 22)

In [4]:
df= df[df['Diabetes'] != 1].copy()
df.loc[df['Diabetes'] == 2, 'Diabetes'] = 1
print(df['Diabetes'].value_counts())

0    40175
1     6968
Name: Diabetes, dtype: int64


In [5]:
categorical_columns= ['HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'Diabetes', 'PhysActivity',
                      'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk',
                      'Sex', 'Age', 'Education', 'Income'
                     ]
df[categorical_columns]= df[categorical_columns].astype(str)
df= pd.get_dummies(df, columns= categorical_columns, drop_first= True)

In [6]:
X= df.drop(['HeartDiseaseorAttack'], axis= 1)
y= df['HeartDiseaseorAttack']

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.2, random_state= 15, stratify= y)

continuous_columns = ['BMI', 'GenHlth', 'MentHlth', 'PhysHlth']

from sklearn.preprocessing import StandardScaler
ss= StandardScaler()
X_train= ss.fit_transform(X_train)
X_test= ss.transform(X_test)

In [8]:
from sklearn.calibration import CalibratedClassifierCV

--- LogisticRegression ---

In [9]:
from sklearn.utils.class_weight import compute_class_weight
class_weights= compute_class_weight(class_weight= 'balanced', classes= np.unique(y_train), y= y_train)
class_weights_dict= {i: class_weights[i] for i in range(len(class_weights))}

from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression
lr= LogisticRegression(max_iter= 1000, random_state= 15, class_weight= class_weights_dict)
lr.fit(X_train, y_train)

lr_calibrated= CalibratedClassifierCV(estimator= lr, method= 'sigmoid', cv= 5)
lr_calibrated.fit(X_train, y_train)
y_pred_lr_calibrated = lr_calibrated.predict(X_test)
y_pred_prob_lr_calibrated= lr_calibrated.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_lr_calibrated, labels= lr_calibrated.classes_)
print(classification_report(y_test, y_pred_lr_calibrated, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lr_calibrated, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lr_calibrated))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lr_calibrated))

              precision    recall  f1-score   support

           0       0.91      0.99      0.95      8517
           1       0.54      0.13      0.21       912

    accuracy                           0.90      9429
   macro avg       0.73      0.56      0.58      9429
weighted avg       0.88      0.90      0.88      9429

Precision: 0.5370370370370371
Accuracy: 0.9049740163325909
AUC: 0.8539682760382229


--- DecisionTreeClassifier ---

In [11]:
class_weights= compute_class_weight(class_weight= 'balanced', classes= np.unique(y_train), y= y_train)
class_weights_dict= {i: class_weights[i] for i in range(len(class_weights))}

from sklearn.tree import DecisionTreeClassifier
dt= DecisionTreeClassifier(random_state=15, class_weight= class_weights_dict)
dt.fit(X_train, y_train)
dt_calibrated = CalibratedClassifierCV(estimator= dt, method= 'isotonic', cv= 5)
dt_calibrated.fit(X_train, y_train)
y_pred_dt_calibrated= dt_calibrated.predict(X_test)
y_pred_prob_dt_calibrated= dt_calibrated.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_dt_calibrated, labels= dt_calibrated.classes_)
print(classification_report(y_test, y_pred_dt_calibrated, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_dt_calibrated, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_dt_calibrated))
print('AUC:', roc_auc_score(y_test, y_pred_prob_dt_calibrated))

              precision    recall  f1-score   support

           0       0.90      1.00      0.95      8517
           1       0.00      0.00      0.00       912

    accuracy                           0.90      9429
   macro avg       0.45      0.50      0.47      9429
weighted avg       0.82      0.90      0.86      9429

Precision: 0.0
Accuracy: 0.9032771237671015
AUC: 0.6901864485682917


--- Tuned - DecisionTreeClassifier ---

In [11]:
from sklearn.model_selection import GridSearchCV
param_grid= {
             'max_depth': [None, 10, 20, 30, 40, 50],
             'min_samples_split': [2, 10, 20],
             'min_samples_leaf': [1, 5, 10],
             'max_features': [None, 'sqrt', 'log2'],
             'criterion': ['gini', 'entropy']
            }

gs_dt= GridSearchCV(estimator= dt, param_grid= param_grid, cv= 5, scoring= 'precision')
gs_dt.fit(X_train, y_train)

print("Best Parameters:", gs_dt.best_params_)
print("Best Precision Score:", gs_dt.best_score_)

Best Parameters: {'criterion': 'entropy', 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 10, 'min_samples_split': 2}
Best Precision Score: 0.46569723206086844


In [12]:
tuned_dt= gs_dt.best_estimator_
tuned_dt_calibrated= CalibratedClassifierCV(base_estimator= tuned_dt, method= 'isotonic', cv= 5)
tuned_dt_calibrated.fit(X_train, y_train)
y_pred_tuned_dt_calibrated= tuned_dt_calibrated.predict(X_test)
y_pred_prob_tuned_dt_calibrated= tuned_dt_calibrated.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_dt_calibrated, labels= tuned_dt_calibrated.classes_)
print(classification_report(y_test, y_pred_tuned_dt_calibrated, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_dt_calibrated, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_dt_calibrated))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_dt_calibrated))

C:\Users\User\anaconda3\lib\site-packages\sklearn\calibration.py:321: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.90      1.00      0.95      8517
           1       0.68      0.02      0.03       912

    accuracy                           0.90      9429
   macro avg       0.79      0.51      0.49      9429
weighted avg       0.88      0.90      0.86      9429

Precision: 0.6818181818181818
Accuracy: 0.9041255700498462
AUC: 0.8337333974980895


--- KNeighborsClassifier ---

In [13]:
from sklearn.neighbors import KNeighborsClassifier
knn= KNeighborsClassifier()
knn.fit(X_train, y_train)
knn_calibrated= CalibratedClassifierCV(base_estimator= knn, method= 'sigmoid', cv= 5)
knn_calibrated.fit(X_train, y_train)
y_pred_knn_calibrated= knn_calibrated.predict(X_test)
y_pred_prob_knn_calibrated= knn_calibrated.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_knn_calibrated, labels= knn_calibrated.classes_)
print(classification_report(y_test, y_pred_knn_calibrated, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_knn_calibrated, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_knn_calibrated))
print('AUC:', roc_auc_score(y_test, y_pred_prob_knn_calibrated))

C:\Users\User\anaconda3\lib\site-packages\sklearn\calibration.py:321: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.91      1.00      0.95      8517
           1       0.53      0.03      0.05       912

    accuracy                           0.90      9429
   macro avg       0.72      0.51      0.50      9429
weighted avg       0.87      0.90      0.86      9429

Precision: 0.5319148936170213
Accuracy: 0.9035952911231308
AUC: 0.7383131054712042


--- Tuned - KNeighborsClassifier ---

In [14]:
param_grid= {
             'n_neighbors': np.arange(1,40),
             'weights': ['uniform', 'distance'],
             'metric': ['euclidean', 'manhattan', 'minkowski']
            }

gs_knn= GridSearchCV(estimator= knn, param_grid= param_grid, cv=5, scoring= 'precision')
gs_knn.fit(X_train, y_train)
print("Best Parameters:", gs_knn.best_params_)
print("Best Precision Score:", gs_knn.best_score_)

Best Parameters: {'metric': 'manhattan', 'n_neighbors': 28, 'weights': 'uniform'}
Best Precision Score: 0.6815538847117795


In [15]:
tuned_knn= gs_knn.best_estimator_
tuned_knn_calibrated = CalibratedClassifierCV(base_estimator= tuned_knn, method= 'sigmoid', cv= 5)
tuned_knn_calibrated.fit(X_train, y_train)
y_pred_tuned_knn_calibrated = tuned_knn_calibrated.predict(X_test)
y_pred_prob_tuned_knn_calibrated= tuned_knn_calibrated.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_knn_calibrated, labels= tuned_knn_calibrated.classes_)
print(classification_report(y_test, y_pred_tuned_knn_calibrated, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_knn_calibrated, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_knn_calibrated))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_knn_calibrated))

C:\Users\User\anaconda3\lib\site-packages\sklearn\calibration.py:321: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.91      0.99      0.95      8517
           1       0.52      0.09      0.15       912

    accuracy                           0.90      9429
   macro avg       0.72      0.54      0.55      9429
weighted avg       0.87      0.90      0.87      9429

Precision: 0.5222929936305732
Accuracy: 0.9040195142645031
AUC: 0.8169965538479286


--- GaussianNB ---

In [16]:
from sklearn.naive_bayes import GaussianNB
nb= GaussianNB()
nb.fit(X_train, y_train)
nb_calibrated = CalibratedClassifierCV(base_estimator= nb, method= 'sigmoid', cv= 5)
nb_calibrated.fit(X_train, y_train)
y_pred_nb_calibrated = nb_calibrated.predict(X_test)
y_pred_prob_nb_calibrated= nb_calibrated.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_nb_calibrated, labels= nb_calibrated.classes_)
print(classification_report(y_test, y_pred_nb_calibrated, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_nb_calibrated, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_nb_calibrated))
print('AUC:', roc_auc_score(y_test, y_pred_prob_nb_calibrated))

C:\Users\User\anaconda3\lib\site-packages\sklearn\calibration.py:321: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.90      1.00      0.95      8517
           1       0.00      0.00      0.00       912

    accuracy                           0.90      9429
   macro avg       0.45      0.50      0.47      9429
weighted avg       0.82      0.90      0.86      9429

Precision: 0.0
Accuracy: 0.9032771237671015
AUC: 0.8058358257684837


--- SVM ---

In [17]:
from sklearn.svm import SVC
svc= SVC(kernel= 'rbf',probability= True, gamma= 1, random_state= 15)
svc.fit(X_train, y_train)
svc_calibrated = CalibratedClassifierCV(base_estimator= svc, method= 'sigmoid', cv= 5)
svc_calibrated.fit(X_train, y_train)
y_pred_svc_calibrated= svc_calibrated.predict(X_test)
y_pred_prob_svc_calibrated= svc_calibrated.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_svc_calibrated, labels= svc_calibrated.classes_)
print(classification_report(y_test, y_pred_svc_calibrated, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_svc_calibrated, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_svc_calibrated))
print('AUC:', roc_auc_score(y_test, y_pred_prob_svc_calibrated))

C:\Users\User\anaconda3\lib\site-packages\sklearn\calibration.py:321: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.90      1.00      0.95      8517
           1       0.00      0.00      0.00       912

    accuracy                           0.90      9429
   macro avg       0.45      0.50      0.47      9429
weighted avg       0.82      0.90      0.86      9429

Precision: 0.0
Accuracy: 0.9028529006257291
AUC: 0.7000333054221793


--- Random Forest ---

In [18]:
from sklearn.ensemble import RandomForestClassifier
rf= RandomForestClassifier(random_state= 15)
rf.fit(X_train, y_train)
rf_calibrated= CalibratedClassifierCV(base_estimator= rf, method= 'isotonic', cv= 5)
rf_calibrated.fit(X_train, y_train)
y_pred_rf_calibrated = rf_calibrated.predict(X_test)
y_pred_prob_rf_calibrated= rf_calibrated.predict_proba(X_test)[:,1]

C:\Users\User\anaconda3\lib\site-packages\sklearn\calibration.py:321: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


In [19]:
cm= confusion_matrix(y_test, y_pred_rf_calibrated, labels= rf_calibrated.classes_)
print(classification_report(y_test, y_pred_rf_calibrated, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_rf_calibrated, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_rf_calibrated))
print('AUC:', roc_auc_score(y_test, y_pred_prob_rf_calibrated))

              precision    recall  f1-score   support

           0       0.90      1.00      0.95      8517
           1       0.62      0.01      0.03       912

    accuracy                           0.90      9429
   macro avg       0.76      0.51      0.49      9429
weighted avg       0.88      0.90      0.86      9429

Precision: 0.6190476190476191
Accuracy: 0.903807402693817
AUC: 0.8288192062726971


In [20]:
param_grid= {
             'n_estimators': [100, 200, 300],
             'max_depth': [None, 10, 20, 30],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
            }

gs_tuned_rf= GridSearchCV(estimator= rf, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_tuned_rf.fit(X_train, y_train)
print("Best Parameters:", gs_tuned_rf.best_params_)
print("Best Precision Score:", gs_tuned_rf.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 300}
Best Precision Score: 0.6932043515080452


In [21]:
tuned_rf= gs_tuned_rf.best_estimator_
tuned_rf_calibrated = CalibratedClassifierCV(base_estimator=tuned_rf, method= 'isotonic', cv= 5)
tuned_rf_calibrated.fit(X_train, y_train)
y_pred_tuned_rf_calibrated = tuned_rf_calibrated.predict(X_test)
y_pred_prob_tuned_rf_calibrated= tuned_rf_calibrated.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_rf_calibrated, labels= tuned_rf_calibrated.classes_)
print(classification_report(y_test, y_pred_tuned_rf_calibrated, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_rf_calibrated, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_rf_calibrated))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_rf_calibrated))

C:\Users\User\anaconda3\lib\site-packages\sklearn\calibration.py:321: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.91      1.00      0.95      8517
           1       0.60      0.05      0.09       912

    accuracy                           0.90      9429
   macro avg       0.75      0.52      0.52      9429
weighted avg       0.88      0.90      0.87      9429

Precision: 0.6027397260273972
Accuracy: 0.9048679605472478
AUC: 0.8498702414572301


--- AdaBoost ---

In [22]:
from sklearn.ensemble import AdaBoostClassifier
ada= AdaBoostClassifier(random_state= 15)
ada.fit(X_train, y_train)
ada_calibrated= CalibratedClassifierCV(base_estimator= ada, method= 'isotonic', cv= 5)
ada_calibrated.fit(X_train, y_train)
y_pred_ada_calibrated = ada_calibrated.predict(X_test)
y_pred_prob_ada_calibrated= ada_calibrated.predict_proba(X_test)[:,1]

C:\Users\User\anaconda3\lib\site-packages\sklearn\calibration.py:321: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


In [23]:
cm= confusion_matrix(y_test, y_pred_ada_calibrated, labels= ada_calibrated.classes_)
print(classification_report(y_test, y_pred_ada_calibrated, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_ada_calibrated, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_ada_calibrated))
print('AUC:', roc_auc_score(y_test, y_pred_prob_ada_calibrated))

              precision    recall  f1-score   support

           0       0.91      0.99      0.95      8517
           1       0.57      0.10      0.17       912

    accuracy                           0.91      9429
   macro avg       0.74      0.55      0.56      9429
weighted avg       0.88      0.91      0.87      9429

Precision: 0.5660377358490566
Accuracy: 0.9055042952593064
AUC: 0.8529882314833697


In [24]:
param_grid= {'n_estimators': [50, 100, 200]}

gs_ada= GridSearchCV(estimator= ada, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_ada.fit(X_train, y_train)
print("Best Parameters:", gs_ada.best_params_)
print("Best Precision Score:", gs_ada.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'n_estimators': 100}
Best Precision Score: 0.5390309313775772


In [25]:
tuned_ada= gs_ada.best_estimator_
tuned_ada_calibrated = CalibratedClassifierCV(base_estimator=tuned_ada, method= 'isotonic', cv= 5)
tuned_ada_calibrated.fit(X_train, y_train)
y_pred_tuned_ada_calibrated= tuned_ada_calibrated.predict(X_test)
y_pred_prob_tuned_ada_calibrated= tuned_ada_calibrated.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_ada_calibrated, labels= tuned_ada_calibrated.classes_)
print(classification_report(y_test, y_pred_tuned_ada_calibrated, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_ada_calibrated, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_ada_calibrated))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_ada_calibrated))

C:\Users\User\anaconda3\lib\site-packages\sklearn\calibration.py:321: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.91      0.99      0.95      8517
           1       0.56      0.09      0.15       912

    accuracy                           0.91      9429
   macro avg       0.74      0.54      0.55      9429
weighted avg       0.88      0.91      0.87      9429

Precision: 0.5642857142857143
Accuracy: 0.9051861279032771
AUC: 0.8526428180790122


--- GradientBoosting ---

In [26]:
from sklearn.ensemble import GradientBoostingClassifier
grb= GradientBoostingClassifier(random_state= 15)
grb.fit(X_train, y_train)
grb_calibrated = CalibratedClassifierCV(base_estimator= grb, method= 'isotonic', cv= 5)
grb_calibrated.fit(X_train, y_train)
y_pred_grb_calibrated= grb_calibrated.predict(X_test)
y_pred_prob_grb_calibrated= grb_calibrated.predict_proba(X_test)[:,1]


cm= confusion_matrix(y_test, y_pred_grb_calibrated, labels= grb_calibrated.classes_)
print(classification_report(y_test, y_pred_grb_calibrated, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_grb_calibrated, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_grb_calibrated))
print('AUC:', roc_auc_score(y_test, y_pred_prob_grb_calibrated))

C:\Users\User\anaconda3\lib\site-packages\sklearn\calibration.py:321: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.91      0.99      0.95      8517
           1       0.58      0.08      0.15       912

    accuracy                           0.91      9429
   macro avg       0.75      0.54      0.55      9429
weighted avg       0.88      0.91      0.87      9429

Precision: 0.5833333333333334
Accuracy: 0.9056103510446495
AUC: 0.8543046775386276


In [27]:
param_grid= {'learning_rate': [0.01, 0.1, 0.2]}

gs_grb= GridSearchCV(estimator= grb, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_grb.fit(X_train, y_train)

print("Best Parameters:", gs_grb.best_params_)
print("Best Precision Score:", gs_grb.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.1}
Best Precision Score: 0.5509480931292992


In [28]:
tuned_grb= gs_grb.best_estimator_
tuned_grb_calibrated = CalibratedClassifierCV(base_estimator= tuned_grb, method= 'isotonic', cv= 5)
tuned_grb_calibrated.fit(X_train, y_train)
y_pred_tuned_grb_calibrated= tuned_grb_calibrated.predict(X_test)
y_pred_prob_tuned_grb_calibrated= tuned_grb_calibrated.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_grb_calibrated, labels= tuned_grb_calibrated.classes_)
print(classification_report(y_test, y_pred_tuned_grb_calibrated, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_grb_calibrated, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_grb_calibrated))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_grb_calibrated))

C:\Users\User\anaconda3\lib\site-packages\sklearn\calibration.py:321: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.91      0.99      0.95      8517
           1       0.58      0.08      0.15       912

    accuracy                           0.91      9429
   macro avg       0.75      0.54      0.55      9429
weighted avg       0.88      0.91      0.87      9429

Precision: 0.5833333333333334
Accuracy: 0.9056103510446495
AUC: 0.8543046775386276


--- XGB ---

In [29]:
from xgboost import XGBClassifier
xgb= XGBClassifier(random_state= 15)
xgb.fit(X_train, y_train)
xgb_calibrated= CalibratedClassifierCV(base_estimator= xgb, method= 'isotonic', cv= 5)
xgb_calibrated.fit(X_train, y_train)
y_pred_xgb_calibrated= xgb_calibrated.predict(X_test)
y_pred_prob_xgb_calibrated= xgb_calibrated.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_xgb_calibrated, labels= xgb_calibrated.classes_)
print(classification_report(y_test, y_pred_xgb_calibrated, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_xgb_calibrated, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_xgb_calibrated))
print('AUC:', roc_auc_score(y_test, y_pred_prob_xgb_calibrated))

C:\Users\User\anaconda3\lib\site-packages\sklearn\calibration.py:321: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.91      1.00      0.95      8517
           1       0.67      0.03      0.05       912

    accuracy                           0.90      9429
   macro avg       0.79      0.51      0.50      9429
weighted avg       0.88      0.90      0.86      9429

Precision: 0.6666666666666666
Accuracy: 0.9046558489765617
AUC: 0.8400571792431648


In [30]:
params_XGBoost= {'learning_rate': [0.01, 0.1, 1.0]}

gs_xgb= GridSearchCV(estimator= xgb, param_grid= params_XGBoost, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_xgb.fit(X_train, y_train)

print("Best Parameters:", gs_xgb.best_params_)
print("Best Precision Score:", gs_xgb.best_score_)

tuned_xgb= gs_xgb.best_estimator_
tuned_xgb_calibrated= CalibratedClassifierCV(base_estimator= tuned_xgb, method= 'isotonic', cv= 5)
tuned_xgb_calibrated.fit(X_train, y_train)
y_pred_tuned_xgb_calibrated= tuned_xgb_calibrated.predict(X_test)
y_pred_prob_tuned_xgb_calibrated= tuned_xgb_calibrated.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_xgb_calibrated, labels= tuned_xgb_calibrated.classes_)
print(classification_report(y_test, y_pred_tuned_xgb_calibrated, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_xgb_calibrated, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_xgb_calibrated))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_xgb_calibrated))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.1}
Best Precision Score: 0.5129835358709718


C:\Users\User\anaconda3\lib\site-packages\sklearn\calibration.py:321: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.91      1.00      0.95      8517
           1       0.58      0.05      0.09       912

    accuracy                           0.90      9429
   macro avg       0.74      0.52      0.52      9429
weighted avg       0.88      0.90      0.87      9429

Precision: 0.5822784810126582
Accuracy: 0.9046558489765617
AUC: 0.8517069962242698


--- LGBM ---

In [31]:
from lightgbm import LGBMClassifier
lgm= LGBMClassifier(random_state= 15)
lgm.fit(X_train, y_train)
lgm_calibrated= CalibratedClassifierCV(base_estimator= lgm, method= 'isotonic', cv= 5)
lgm_calibrated.fit(X_train, y_train)
y_pred_lgm_calibrated= lgm_calibrated.predict(X_test)
y_pred_prob_lgm_calibrated= lgm_calibrated.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_lgm_calibrated, labels= lgm_calibrated.classes_)
print(classification_report(y_test, y_pred_lgm_calibrated, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lgm_calibrated, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lgm_calibrated))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lgm_calibrated))

[LightGBM] [Info] Number of positive: 3649, number of negative: 34065
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004215 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 251
[LightGBM] [Info] Number of data points in the train set: 37714, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.096755 -> initscore=-2.233817
[LightGBM] [Info] Start training from score -2.233817


C:\Users\User\anaconda3\lib\site-packages\sklearn\calibration.py:321: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


[LightGBM] [Info] Number of positive: 2919, number of negative: 27252
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006287 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 250
[LightGBM] [Info] Number of data points in the train set: 30171, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.096749 -> initscore=-2.233886
[LightGBM] [Info] Start training from score -2.233886
[LightGBM] [Info] Number of positive: 2919, number of negative: 27252
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003521 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 250
[LightGBM] [Info] Number of data points in the train set: 30171, number of used features: 42
[LightGBM] [Info] [bin

In [32]:
params_LGB= {'learning_rate': [0.001, 0.01, 0.1, 1.0],
             'num_leaves': [31, 127],
             'reg_alpha': [0.1, 0.5],
             'min_data_in_leaf': [30, 50, 100, 300, 400]}

gs_lgm= GridSearchCV(estimator= lgm, param_grid= params_LGB, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_lgm.fit(X_train, y_train)

print("Best Parameters:", gs_lgm.best_params_)
print("Best Precision Score:", gs_lgm.best_score_)

tuned_lgm= gs_lgm.best_estimator_
tuned_lgm_calibrated = CalibratedClassifierCV(base_estimator=tuned_lgm, method= 'isotonic', cv= 5)
tuned_lgm_calibrated.fit(X_train, y_train)
y_pred_tuned_lgm_calibrated= tuned_lgm_calibrated.predict(X_test)
y_pred_prob_tuned_lgm_calibrated= tuned_lgm_calibrated.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_lgm_calibrated, labels= tuned_lgm_calibrated.classes_)
print(classification_report(y_test, y_pred_tuned_lgm_calibrated, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_lgm_calibrated, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_lgm_calibrated))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_lgm_calibrated))

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Info] Number of positive: 3649, number of negative: 34065
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011599 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 251
[LightGBM] [Info] Number of data points in the train set: 37714, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.096755 -> initscore=-2.233817
[LightGBM] [Info] Start training from score -2.233817
Best Parameters: {'learning_rate': 0.01, 'min_data_in_leaf': 30, 'num_leaves': 127, 'reg_alpha': 0.1}
Best Precision Score: 0.6
[LightGBM]

C:\Users\User\anaconda3\lib\site-packages\sklearn\calibration.py:321: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Info] Number of positive: 2919, number of negative: 27252
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006668 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 250
[LightGBM] [Info] Number of data points in the train set: 30171, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.096749 -> initscore=-2.233886
[LightGBM] [Info] Start training from score -2.233886
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be 

--- CatBoost ---

In [33]:
from catboost import CatBoostClassifier
cat= CatBoostClassifier(random_state= 15)
cat.fit(X_train, y_train)
cat_calibrated = CalibratedClassifierCV(base_estimator= cat, method= 'isotonic', cv= 5)
cat_calibrated.fit(X_train, y_train)
y_pred_cat_calibrated= cat_calibrated.predict(X_test)
y_pred_prob_cat_calibrated= cat_calibrated.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_cat_calibrated, labels= cat_calibrated.classes_)
print(classification_report(y_test, y_pred_cat_calibrated, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_cat_calibrated, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_cat_calibrated))
print('AUC:', roc_auc_score(y_test, y_pred_prob_cat_calibrated))

Learning rate set to 0.04854
0:	learn: 0.6390849	total: 157ms	remaining: 2m 37s
1:	learn: 0.5937888	total: 168ms	remaining: 1m 23s
2:	learn: 0.5573397	total: 178ms	remaining: 59s
3:	learn: 0.5183734	total: 189ms	remaining: 47s
4:	learn: 0.4915505	total: 202ms	remaining: 40.2s
5:	learn: 0.4629431	total: 214ms	remaining: 35.4s
6:	learn: 0.4394643	total: 226ms	remaining: 32s
7:	learn: 0.4203563	total: 234ms	remaining: 29.1s
8:	learn: 0.4018322	total: 243ms	remaining: 26.8s
9:	learn: 0.3857597	total: 252ms	remaining: 24.9s
10:	learn: 0.3718986	total: 259ms	remaining: 23.3s
11:	learn: 0.3580574	total: 267ms	remaining: 22s
12:	learn: 0.3481623	total: 274ms	remaining: 20.8s
13:	learn: 0.3389341	total: 282ms	remaining: 19.9s
14:	learn: 0.3310245	total: 290ms	remaining: 19s
15:	learn: 0.3232157	total: 298ms	remaining: 18.3s
16:	learn: 0.3161048	total: 306ms	remaining: 17.7s
17:	learn: 0.3109163	total: 314ms	remaining: 17.1s
18:	learn: 0.3049972	total: 322ms	remaining: 16.6s
19:	learn: 0.2999864

175:	learn: 0.2354213	total: 1.76s	remaining: 8.25s
176:	learn: 0.2353368	total: 1.78s	remaining: 8.26s
177:	learn: 0.2352342	total: 1.78s	remaining: 8.24s
178:	learn: 0.2351318	total: 1.79s	remaining: 8.22s
179:	learn: 0.2350745	total: 1.8s	remaining: 8.21s
180:	learn: 0.2350074	total: 1.81s	remaining: 8.2s
181:	learn: 0.2349453	total: 1.82s	remaining: 8.18s
182:	learn: 0.2348808	total: 1.83s	remaining: 8.16s
183:	learn: 0.2348103	total: 1.84s	remaining: 8.16s
184:	learn: 0.2347510	total: 1.85s	remaining: 8.14s
185:	learn: 0.2346752	total: 1.86s	remaining: 8.12s
186:	learn: 0.2345954	total: 1.86s	remaining: 8.11s
187:	learn: 0.2345258	total: 1.87s	remaining: 8.09s
188:	learn: 0.2344432	total: 1.88s	remaining: 8.08s
189:	learn: 0.2343602	total: 1.89s	remaining: 8.06s
190:	learn: 0.2342729	total: 1.9s	remaining: 8.05s
191:	learn: 0.2341721	total: 1.91s	remaining: 8.04s
192:	learn: 0.2340644	total: 1.92s	remaining: 8.02s
193:	learn: 0.2340067	total: 1.93s	remaining: 8.01s
194:	learn: 0.2

338:	learn: 0.2235874	total: 3.57s	remaining: 6.97s
339:	learn: 0.2235293	total: 3.59s	remaining: 6.96s
340:	learn: 0.2234927	total: 3.6s	remaining: 6.96s
341:	learn: 0.2234355	total: 3.62s	remaining: 6.96s
342:	learn: 0.2233708	total: 3.63s	remaining: 6.96s
343:	learn: 0.2232997	total: 3.65s	remaining: 6.95s
344:	learn: 0.2232702	total: 3.66s	remaining: 6.95s
345:	learn: 0.2232165	total: 3.67s	remaining: 6.94s
346:	learn: 0.2231578	total: 3.69s	remaining: 6.94s
347:	learn: 0.2230950	total: 3.7s	remaining: 6.93s
348:	learn: 0.2230374	total: 3.71s	remaining: 6.93s
349:	learn: 0.2229819	total: 3.73s	remaining: 6.93s
350:	learn: 0.2228895	total: 3.74s	remaining: 6.92s
351:	learn: 0.2228344	total: 3.76s	remaining: 6.92s
352:	learn: 0.2227665	total: 3.77s	remaining: 6.91s
353:	learn: 0.2226898	total: 3.79s	remaining: 6.91s
354:	learn: 0.2226039	total: 3.8s	remaining: 6.91s
355:	learn: 0.2225421	total: 3.82s	remaining: 6.91s
356:	learn: 0.2224907	total: 3.84s	remaining: 6.92s
357:	learn: 0.2

497:	learn: 0.2145578	total: 5.95s	remaining: 6s
498:	learn: 0.2145038	total: 5.96s	remaining: 5.98s
499:	learn: 0.2144487	total: 5.96s	remaining: 5.96s
500:	learn: 0.2144097	total: 5.97s	remaining: 5.95s
501:	learn: 0.2143554	total: 5.98s	remaining: 5.93s
502:	learn: 0.2142778	total: 5.99s	remaining: 5.92s
503:	learn: 0.2142318	total: 6s	remaining: 5.9s
504:	learn: 0.2141712	total: 6.01s	remaining: 5.89s
505:	learn: 0.2140990	total: 6.02s	remaining: 5.87s
506:	learn: 0.2140429	total: 6.02s	remaining: 5.86s
507:	learn: 0.2139798	total: 6.03s	remaining: 5.84s
508:	learn: 0.2139008	total: 6.04s	remaining: 5.83s
509:	learn: 0.2138394	total: 6.05s	remaining: 5.81s
510:	learn: 0.2137767	total: 6.06s	remaining: 5.8s
511:	learn: 0.2137081	total: 6.07s	remaining: 5.78s
512:	learn: 0.2136803	total: 6.08s	remaining: 5.77s
513:	learn: 0.2136320	total: 6.08s	remaining: 5.75s
514:	learn: 0.2136034	total: 6.09s	remaining: 5.74s
515:	learn: 0.2135453	total: 6.1s	remaining: 5.72s
516:	learn: 0.2134930

671:	learn: 0.2059411	total: 7.52s	remaining: 3.67s
672:	learn: 0.2058964	total: 7.54s	remaining: 3.66s
673:	learn: 0.2058551	total: 7.55s	remaining: 3.65s
674:	learn: 0.2058132	total: 7.56s	remaining: 3.64s
675:	learn: 0.2057542	total: 7.57s	remaining: 3.63s
676:	learn: 0.2057177	total: 7.58s	remaining: 3.61s
677:	learn: 0.2056693	total: 7.58s	remaining: 3.6s
678:	learn: 0.2056041	total: 7.59s	remaining: 3.59s
679:	learn: 0.2055539	total: 7.6s	remaining: 3.58s
680:	learn: 0.2054862	total: 7.61s	remaining: 3.56s
681:	learn: 0.2054566	total: 7.62s	remaining: 3.55s
682:	learn: 0.2054177	total: 7.63s	remaining: 3.54s
683:	learn: 0.2053812	total: 7.64s	remaining: 3.53s
684:	learn: 0.2053489	total: 7.64s	remaining: 3.52s
685:	learn: 0.2052943	total: 7.65s	remaining: 3.5s
686:	learn: 0.2052438	total: 7.66s	remaining: 3.49s
687:	learn: 0.2051981	total: 7.67s	remaining: 3.48s
688:	learn: 0.2051440	total: 7.68s	remaining: 3.46s
689:	learn: 0.2051064	total: 7.69s	remaining: 3.45s
690:	learn: 0.2

...

227:	learn: 0.2265592	total: 2.58s	remaining: 8.72s
228:	learn: 0.2264734	total: 2.59s	remaining: 8.71s
229:	learn: 0.2263821	total: 2.6s	remaining: 8.7s
230:	learn: 0.2263163	total: 2.61s	remaining: 8.69s
231:	learn: 0.2262353	total: 2.62s	remaining: 8.68s
232:	learn: 0.2260957	total: 2.63s	remaining: 8.68s
233:	learn: 0.2260355	total: 2.65s	remaining: 8.67s
234:	learn: 0.2259855	total: 2.66s	remaining: 8.65s
235:	learn: 0.2258885	total: 2.67s	remaining: 8.64s
236:	learn: 0.2258382	total: 2.68s	remaining: 8.63s
237:	learn: 0.2257512	total: 2.69s	remaining: 8.61s
238:	learn: 0.2257075	total: 2.7s	remaining: 8.6s
239:	learn: 0.2256556	total: 2.71s	remaining: 8.59s
240:	learn: 0.2255766	total: 2.72s	remaining: 8.58s
241:	learn: 0.2254953	total: 2.73s	remaining: 8.56s
242:	learn: 0.2254013	total: 2.74s	remaining: 8.55s
243:	learn: 0.2253297	total: 2.75s	remaining: 8.53s
244:	learn: 0.2252825	total: 2.77s	remaining: 8.54s
245:	learn: 0.2251719	total: 2.78s	remaining: 8.52s
246:	learn: 0.22

399:	learn: 0.2135438	total: 4.15s	remaining: 6.22s
400:	learn: 0.2134330	total: 4.16s	remaining: 6.21s
401:	learn: 0.2133618	total: 4.17s	remaining: 6.2s
402:	learn: 0.2132733	total: 4.18s	remaining: 6.19s
403:	learn: 0.2132086	total: 4.18s	remaining: 6.17s
404:	learn: 0.2131470	total: 4.19s	remaining: 6.16s
405:	learn: 0.2130839	total: 4.2s	remaining: 6.14s
406:	learn: 0.2130098	total: 4.21s	remaining: 6.14s
407:	learn: 0.2129501	total: 4.22s	remaining: 6.13s
408:	learn: 0.2128962	total: 4.23s	remaining: 6.11s
409:	learn: 0.2128214	total: 4.24s	remaining: 6.1s
410:	learn: 0.2127238	total: 4.25s	remaining: 6.09s
411:	learn: 0.2126530	total: 4.26s	remaining: 6.08s
412:	learn: 0.2126011	total: 4.27s	remaining: 6.06s
413:	learn: 0.2125624	total: 4.27s	remaining: 6.05s
414:	learn: 0.2124799	total: 4.28s	remaining: 6.04s
415:	learn: 0.2124331	total: 4.29s	remaining: 6.02s
416:	learn: 0.2123710	total: 4.3s	remaining: 6.01s
417:	learn: 0.2122914	total: 4.31s	remaining: 6s
418:	learn: 0.21225

577:	learn: 0.2019603	total: 5.69s	remaining: 4.16s
578:	learn: 0.2019118	total: 5.7s	remaining: 4.14s
579:	learn: 0.2018738	total: 5.71s	remaining: 4.13s
580:	learn: 0.2018388	total: 5.72s	remaining: 4.12s
581:	learn: 0.2017912	total: 5.72s	remaining: 4.11s
582:	learn: 0.2017330	total: 5.73s	remaining: 4.1s
583:	learn: 0.2016557	total: 5.74s	remaining: 4.09s
584:	learn: 0.2015971	total: 5.75s	remaining: 4.08s
585:	learn: 0.2015251	total: 5.76s	remaining: 4.07s
586:	learn: 0.2014676	total: 5.76s	remaining: 4.06s
587:	learn: 0.2014226	total: 5.77s	remaining: 4.04s
588:	learn: 0.2013757	total: 5.78s	remaining: 4.03s
589:	learn: 0.2013373	total: 5.79s	remaining: 4.02s
590:	learn: 0.2012790	total: 5.8s	remaining: 4.01s
591:	learn: 0.2012185	total: 5.8s	remaining: 4s
592:	learn: 0.2011776	total: 5.81s	remaining: 3.99s
593:	learn: 0.2011249	total: 5.82s	remaining: 3.98s
594:	learn: 0.2010878	total: 5.83s	remaining: 3.97s
595:	learn: 0.2010269	total: 5.84s	remaining: 3.96s
596:	learn: 0.20096

758:	learn: 0.1928718	total: 7.24s	remaining: 2.3s
759:	learn: 0.1928224	total: 7.25s	remaining: 2.29s
760:	learn: 0.1927676	total: 7.26s	remaining: 2.28s
761:	learn: 0.1927406	total: 7.27s	remaining: 2.27s
762:	learn: 0.1927002	total: 7.27s	remaining: 2.26s
763:	learn: 0.1926825	total: 7.28s	remaining: 2.25s
764:	learn: 0.1926194	total: 7.29s	remaining: 2.24s
765:	learn: 0.1925918	total: 7.3s	remaining: 2.23s
766:	learn: 0.1925557	total: 7.31s	remaining: 2.22s
767:	learn: 0.1925511	total: 7.31s	remaining: 2.21s
768:	learn: 0.1925216	total: 7.32s	remaining: 2.2s
769:	learn: 0.1924678	total: 7.33s	remaining: 2.19s
770:	learn: 0.1924103	total: 7.34s	remaining: 2.18s
771:	learn: 0.1923825	total: 7.35s	remaining: 2.17s
772:	learn: 0.1923122	total: 7.35s	remaining: 2.16s
773:	learn: 0.1922595	total: 7.36s	remaining: 2.15s
774:	learn: 0.1922082	total: 7.37s	remaining: 2.14s
775:	learn: 0.1921715	total: 7.38s	remaining: 2.13s
776:	learn: 0.1921335	total: 7.38s	remaining: 2.12s
777:	learn: 0.1

918:	learn: 0.1856679	total: 8.58s	remaining: 756ms
919:	learn: 0.1856168	total: 8.59s	remaining: 747ms
920:	learn: 0.1855596	total: 8.6s	remaining: 738ms
921:	learn: 0.1855144	total: 8.61s	remaining: 728ms
922:	learn: 0.1854711	total: 8.61s	remaining: 719ms
923:	learn: 0.1854338	total: 8.62s	remaining: 709ms
924:	learn: 0.1853673	total: 8.63s	remaining: 700ms
925:	learn: 0.1853159	total: 8.64s	remaining: 690ms
926:	learn: 0.1852626	total: 8.64s	remaining: 681ms
927:	learn: 0.1852062	total: 8.65s	remaining: 671ms
928:	learn: 0.1851618	total: 8.66s	remaining: 662ms
929:	learn: 0.1851143	total: 8.67s	remaining: 652ms
930:	learn: 0.1850701	total: 8.68s	remaining: 643ms
931:	learn: 0.1850326	total: 8.68s	remaining: 634ms
932:	learn: 0.1850014	total: 8.69s	remaining: 624ms
933:	learn: 0.1849661	total: 8.7s	remaining: 615ms
934:	learn: 0.1849232	total: 8.71s	remaining: 605ms
935:	learn: 0.1848824	total: 8.72s	remaining: 596ms
936:	learn: 0.1848501	total: 8.72s	remaining: 587ms
937:	learn: 0.

In [34]:
params_CatBoost= {
                  'depth': [3,5,10],
                  'learning_rate' : [0.01,0.1,1],
                  'iterations' : [5,10,50,100]
                 }

gs_cat= GridSearchCV(estimator= cat, param_grid= params_CatBoost, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_cat.fit(X_train, y_train)

print("Best Parameters:", gs_cat.best_params_)
print("Best Precision Score:", gs_cat.best_score_)

tuned_cat= gs_cat.best_estimator_
tuned_cat_calibrated = CalibratedClassifierCV(base_estimator= tuned_cat, method= 'isotonic', cv= 5)
tuned_cat_calibrated.fit(X_train, y_train)
y_pred_tuned_cat_calibrated= tuned_cat_calibrated.predict(X_test)
y_pred_prob_tuned_cat_calibrated= tuned_cat_calibrated.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_cat_calibrated, labels= tuned_cat_calibrated.classes_)
print(classification_report(y_test, y_pred_tuned_cat_calibrated, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_cat_calibrated, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_cat_calibrated))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_cat_calibrated))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
0:	learn: 0.6816196	total: 9.63ms	remaining: 953ms
1:	learn: 0.6714722	total: 19.1ms	remaining: 937ms
2:	learn: 0.6613995	total: 27.2ms	remaining: 881ms
3:	learn: 0.6513357	total: 37.2ms	remaining: 892ms
4:	learn: 0.6413585	total: 45.3ms	remaining: 861ms
5:	learn: 0.6312247	total: 54.6ms	remaining: 855ms
6:	learn: 0.6222469	total: 64ms	remaining: 850ms
7:	learn: 0.6136571	total: 73.1ms	remaining: 840ms
8:	learn: 0.6052308	total: 81.5ms	remaining: 824ms
9:	learn: 0.5963060	total: 89.2ms	remaining: 803ms
10:	learn: 0.5880142	total: 98ms	remaining: 793ms
11:	learn: 0.5800307	total: 106ms	remaining: 780ms
12:	learn: 0.5718503	total: 115ms	remaining: 772ms
13:	learn: 0.5638283	total: 125ms	remaining: 767ms
14:	learn: 0.5561548	total: 134ms	remaining: 759ms
15:	learn: 0.5482328	total: 145ms	remaining: 761ms
16:	learn: 0.5406864	total: 154ms	remaining: 754ms
17:	learn: 0.5335859	total: 163ms	remaining: 741ms
18:	learn: 0.5271269	to

C:\Users\User\anaconda3\lib\site-packages\sklearn\calibration.py:321: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


0:	learn: 0.6817891	total: 9.49ms	remaining: 940ms
1:	learn: 0.6716214	total: 18ms	remaining: 884ms
2:	learn: 0.6618499	total: 26.5ms	remaining: 857ms
3:	learn: 0.6517651	total: 35ms	remaining: 841ms
4:	learn: 0.6425055	total: 40.1ms	remaining: 763ms
5:	learn: 0.6331179	total: 47.5ms	remaining: 744ms
6:	learn: 0.6245030	total: 56ms	remaining: 744ms
7:	learn: 0.6152932	total: 63.1ms	remaining: 725ms
8:	learn: 0.6062323	total: 70.7ms	remaining: 715ms
9:	learn: 0.5980083	total: 78.6ms	remaining: 708ms
10:	learn: 0.5902262	total: 86.3ms	remaining: 698ms
11:	learn: 0.5822962	total: 94.1ms	remaining: 690ms
12:	learn: 0.5739329	total: 102ms	remaining: 681ms
13:	learn: 0.5662207	total: 110ms	remaining: 677ms
14:	learn: 0.5595056	total: 119ms	remaining: 674ms
15:	learn: 0.5515863	total: 127ms	remaining: 664ms
16:	learn: 0.5446990	total: 135ms	remaining: 658ms
17:	learn: 0.5378905	total: 145ms	remaining: 661ms
18:	learn: 0.5316139	total: 153ms	remaining: 653ms
19:	learn: 0.5251369	total: 160ms	r

82:	learn: 0.3215472	total: 572ms	remaining: 117ms
83:	learn: 0.3202255	total: 580ms	remaining: 111ms
84:	learn: 0.3187798	total: 589ms	remaining: 104ms
85:	learn: 0.3175227	total: 596ms	remaining: 97ms
86:	learn: 0.3163474	total: 603ms	remaining: 90.2ms
87:	learn: 0.3151743	total: 609ms	remaining: 83.1ms
88:	learn: 0.3138010	total: 617ms	remaining: 76.3ms
89:	learn: 0.3127244	total: 624ms	remaining: 69.4ms
90:	learn: 0.3116982	total: 632ms	remaining: 62.5ms
91:	learn: 0.3105801	total: 639ms	remaining: 55.5ms
92:	learn: 0.3094898	total: 646ms	remaining: 48.6ms
93:	learn: 0.3086565	total: 652ms	remaining: 41.6ms
94:	learn: 0.3074931	total: 658ms	remaining: 34.6ms
95:	learn: 0.3065995	total: 666ms	remaining: 27.7ms
96:	learn: 0.3055046	total: 672ms	remaining: 20.8ms
97:	learn: 0.3044404	total: 679ms	remaining: 13.9ms
98:	learn: 0.3035715	total: 685ms	remaining: 6.92ms
99:	learn: 0.3027666	total: 692ms	remaining: 0us
0:	learn: 0.6817186	total: 11.2ms	remaining: 1.11s
1:	learn: 0.6717039	t

53:	learn: 0.3771507	total: 374ms	remaining: 319ms
54:	learn: 0.3740885	total: 383ms	remaining: 313ms
55:	learn: 0.3715515	total: 389ms	remaining: 306ms
56:	learn: 0.3688400	total: 396ms	remaining: 299ms
57:	learn: 0.3664993	total: 402ms	remaining: 291ms
58:	learn: 0.3640661	total: 409ms	remaining: 284ms
59:	learn: 0.3619030	total: 415ms	remaining: 277ms
60:	learn: 0.3595189	total: 422ms	remaining: 270ms
61:	learn: 0.3574341	total: 428ms	remaining: 262ms
62:	learn: 0.3554219	total: 434ms	remaining: 255ms
63:	learn: 0.3532458	total: 441ms	remaining: 248ms
64:	learn: 0.3510005	total: 447ms	remaining: 241ms
65:	learn: 0.3492921	total: 453ms	remaining: 233ms
66:	learn: 0.3473128	total: 460ms	remaining: 226ms
67:	learn: 0.3453158	total: 466ms	remaining: 219ms
68:	learn: 0.3437111	total: 473ms	remaining: 212ms
69:	learn: 0.3417015	total: 479ms	remaining: 205ms
70:	learn: 0.3399605	total: 486ms	remaining: 198ms
71:	learn: 0.3381321	total: 492ms	remaining: 191ms
72:	learn: 0.3364786	total: 498

--- Visualisation ---

In [35]:
precision_scores= {
                    'Logistic Regression Precision:': precision_score(y_test, y_pred_lr_calibrated, zero_division= 0),
                    'Decision Tree Precision:': precision_score(y_test, y_pred_dt_calibrated, zero_division= 0),
                    'Tuned Decision Tree Precision:': precision_score(y_test, y_pred_tuned_dt_calibrated, zero_division= 0),
                    'KNeighborsClassifier Precision:': precision_score(y_test, y_pred_knn_calibrated, zero_division= 0),
                    'Tuned KNeighborsClassifier Precision:': precision_score(y_test, y_pred_tuned_knn_calibrated, zero_division= 0),
                    'GaussianNB Precision:': precision_score(y_test, y_pred_nb_calibrated, zero_division= 0),
                    'SVM Precision:': precision_score(y_test, y_pred_svc_calibrated, zero_division= 0),
                    'Random Forest Precision:': precision_score(y_test, y_pred_rf_calibrated, zero_division= 0),
                    'Tuned Random Forest Precision:': precision_score(y_test, y_pred_tuned_rf_calibrated, zero_division= 0),
                    'AdaBoost Precision:': precision_score(y_test, y_pred_ada_calibrated, zero_division= 0),
                    'Tuned AdaBoost Precision:': precision_score(y_test, y_pred_tuned_ada_calibrated, zero_division= 0),
                    'GradientBoosting Precision:': precision_score(y_test, y_pred_grb_calibrated, zero_division= 0),
                    'Tuned GradientBoosting Precision:': precision_score(y_test, y_pred_tuned_grb_calibrated, zero_division= 0),
                    'XGB Precision:': precision_score(y_test, y_pred_xgb_calibrated, zero_division= 0),
                    'Tuned XGB Precision:': precision_score(y_test, y_pred_tuned_xgb_calibrated, zero_division= 0),
                    'LGBM Precision:': precision_score(y_test, y_pred_lgm_calibrated, zero_division= 0),
                    'Tuned LGBM Precision:': precision_score(y_test, y_pred_tuned_lgm_calibrated, zero_division= 0),
                    'CatBoost Precision:': precision_score(y_test, y_pred_cat_calibrated, zero_division= 0),
                    'Tuned CatBoost Precision:': precision_score(y_test, y_pred_tuned_cat_calibrated, zero_division= 0)
                  }

calibrated_precision= pd.DataFrame(list(precision_scores.items()), columns= ['Model', 'Precision Score'])
calibrated_precision= calibrated_precision.sort_values(by= 'Precision Score', ascending=False)
print(calibrated_precision)

                                    Model  Precision Score
2          Tuned Decision Tree Precision:         0.681818
13                         XGB Precision:         0.666667
7                Random Forest Precision:         0.619048
16                  Tuned LGBM Precision:         0.604938
8          Tuned Random Forest Precision:         0.602740
11            GradientBoosting Precision:         0.583333
12      Tuned GradientBoosting Precision:         0.583333
14                   Tuned XGB Precision:         0.582278
17                    CatBoost Precision:         0.582090
18              Tuned CatBoost Precision:         0.578125
15                        LGBM Precision:         0.576471
9                     AdaBoost Precision:         0.566038
10              Tuned AdaBoost Precision:         0.564286
0          Logistic Regression Precision:         0.552885
3         KNeighborsClassifier Precision:         0.531915
4   Tuned KNeighborsClassifier Precision:         0.5222

In [36]:
accuracy_scores= {
                    'Logistic Regression Accuracy:': accuracy_score(y_test, y_pred_lr_calibrated),
                    'Decision Tree Accuracy:': accuracy_score(y_test, y_pred_dt_calibrated),
                    'Tuned Decision Tree Accuracy:': accuracy_score(y_test, y_pred_tuned_dt_calibrated),
                    'KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_knn_calibrated),
                    'Tuned KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_tuned_knn_calibrated),
                    'GaussianNB Accuracy:': accuracy_score(y_test, y_pred_nb_calibrated),
                    'SVM Accuracy:': accuracy_score(y_test, y_pred_svc_calibrated),
                    'Random Forest Accuracy:': accuracy_score(y_test, y_pred_rf_calibrated),
                    'Tuned Random Forest Accuracy:': accuracy_score(y_test, y_pred_tuned_rf_calibrated),
                    'AdaBoost Accuracy:': accuracy_score(y_test, y_pred_ada_calibrated),
                    'Tuned AdaBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_ada_calibrated),
                    'GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_grb_calibrated),
                    'Tuned GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_tuned_grb_calibrated),
                    'XGB Accuracy:': accuracy_score(y_test, y_pred_xgb_calibrated),
                    'Tuned XGB Accuracy:': accuracy_score(y_test, y_pred_tuned_xgb_calibrated),
                    'LGBM Accuracy:': accuracy_score(y_test, y_pred_lgm_calibrated),
                    'Tuned LGBM Accuracy:': accuracy_score(y_test, y_pred_tuned_lgm_calibrated),
                    'CatBoost Accuracy:': accuracy_score(y_test, y_pred_cat_calibrated),
                    'Tuned CatBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_cat_calibrated)
                  }

calibrated_accuracy= pd.DataFrame(list(accuracy_scores.items()), columns= ['Model', 'Accuracy Score'])
calibrated_accuracy= calibrated_accuracy.sort_values(by= 'Accuracy Score', ascending=False)
print(calibrated_accuracy)

                                   Model  Accuracy Score
0          Logistic Regression Accuracy:        0.905610
11            GradientBoosting Accuracy:        0.905610
12      Tuned GradientBoosting Accuracy:        0.905610
9                     AdaBoost Accuracy:        0.905504
18              Tuned CatBoost Accuracy:        0.905398
10              Tuned AdaBoost Accuracy:        0.905186
16                  Tuned LGBM Accuracy:        0.905080
8          Tuned Random Forest Accuracy:        0.904868
13                         XGB Accuracy:        0.904656
14                   Tuned XGB Accuracy:        0.904656
15                        LGBM Accuracy:        0.904656
17                    CatBoost Accuracy:        0.904444
2          Tuned Decision Tree Accuracy:        0.904126
4   Tuned KNeighborsClassifier Accuracy:        0.904020
7                Random Forest Accuracy:        0.903807
3         KNeighborsClassifier Accuracy:        0.903595
5                   GaussianNB 

In [37]:
auc_scores= {
                    'Logistic Regression AUC:': roc_auc_score(y_test, y_pred_prob_lr_calibrated),
                    'Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_dt_calibrated),
                    'Tuned Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_tuned_dt_calibrated),
                    'KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_knn_calibrated),
                    'Tuned KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_tuned_knn_calibrated),
                    'GaussianNB AUC:': roc_auc_score(y_test, y_pred_prob_nb_calibrated),
                    'SVM AUC:': roc_auc_score(y_test, y_pred_prob_svc_calibrated),
                    'Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_rf_calibrated),
                    'Tuned Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_tuned_rf_calibrated),
                    'AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_ada_calibrated),
                    'Tuned AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_ada_calibrated),
                    'GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_grb_calibrated),
                    'Tuned GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_tuned_grb_calibrated),
                    'XGB AUC:': roc_auc_score(y_test, y_pred_prob_xgb_calibrated),
                    'Tuned XGB AUC:': roc_auc_score(y_test, y_pred_prob_tuned_xgb_calibrated),
                    'LGBM AUC:': roc_auc_score(y_test, y_pred_prob_lgm_calibrated),
                    'Tuned LGBM AUC:': roc_auc_score(y_test, y_pred_prob_tuned_lgm_calibrated),
                    'CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_cat_calibrated),
                    'Tuned CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_cat_calibrated)
                  }

calibrated_auc= pd.DataFrame(list(auc_scores.items()), columns= ['Model', 'AUC Score'])
calibrated_auc= calibrated_auc.sort_values(by= 'AUC Score', ascending=False)
print(calibrated_auc)

                              Model  AUC Score
0          Logistic Regression AUC:   0.854543
11            GradientBoosting AUC:   0.854305
12      Tuned GradientBoosting AUC:   0.854305
9                     AdaBoost AUC:   0.852988
10              Tuned AdaBoost AUC:   0.852643
14                   Tuned XGB AUC:   0.851707
15                        LGBM AUC:   0.851502
8          Tuned Random Forest AUC:   0.849870
17                    CatBoost AUC:   0.848985
16                  Tuned LGBM AUC:   0.845390
18              Tuned CatBoost AUC:   0.840291
13                         XGB AUC:   0.840057
2          Tuned Decision Tree AUC:   0.833733
7                Random Forest AUC:   0.828819
4   Tuned KNeighborsClassifier AUC:   0.816997
5                   GaussianNB AUC:   0.805836
3         KNeighborsClassifier AUC:   0.738313
1                Decision Tree AUC:   0.709445
6                          SVM AUC:   0.700033
